In [ ]:
# test_rds_permissions.ipynb

import psycopg2

# 1) Connect to the remote RDS
conn = psycopg2.connect(
    host="your_rds_host_here",
    port=5432,
    dbname="your_database_name",
    user="your_db_username",
    password="your_db_password"
)

try:
    # 2) Open a cursor, run SQL commands
    with conn.cursor() as cur:
        # 2a) INSERT a test row in 'matter_category'
        insert_sql = """
            INSERT INTO matter_category (category_type)
            VALUES (%s)
            RETURNING category_id;
        """
        cur.execute(insert_sql, ("test_category_type",))
        new_id = cur.fetchone()[0]
        print("Inserted row in matter_category with category_id =", new_id)

        # 2b) SELECT to confirm it’s there
        select_sql = """
            SELECT category_type
            FROM matter_category
            WHERE category_id = %s;
        """
        cur.execute(select_sql, (new_id,))
        row = cur.fetchone()
        print("Retrieved row:", row)

        # 2c) DELETE to confirm delete permissions
        delete_sql = """
            DELETE FROM matter_category
            WHERE category_id = %s;
        """
        cur.execute(delete_sql, (new_id,))
        print("Deleted row with category_id =", new_id)

    # 3) Commit the changes
    conn.commit()

except Exception as e:
    # 4) Roll back on error
    conn.rollback()
    print("Error occurred. Transaction rolled back.\n", e)

finally:
    # 5) Close the connection
    conn.close()
    print("Connection closed.")
